In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Load the sleep data into a pandas dataframe
df = pd.read_csv('/kk (1).csv')
#dropping NaN values
df2=df.dropna()
# Define the feature variables (X) and target variable (y)
X = df2[['light sleep (sec)', 'deep (sec)', 'start time from 8pm','end time from 8pm']]
y = df2['Snooze']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the logistic regression model to the training data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict the snooze count for the testing data
y_pred = logreg.predict(X_test)

# Print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

ModuleNotFoundError: No module named 'sklearn'